In [1]:
import os
import pathlib
import shutil

import kaggle
import kagglehub
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
import sklearn
import tensorflow as tf

2025-02-07 13:47:29.827682: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Downloading the competition

In [83]:
competition_name = "house-prices-advanced-regression-techniques"  # Change this to the competition you want to download
competition_path = pathlib.Path(
    kagglehub.competition.competition_download(competition_name)
)  # Downloads the competition
competition_files: list = os.listdir(
    competition_path
)  # List of the files included in the competition

print(competition_files)

# copy files to the current directory
shutil.copyfile(
    competition_path / "sample_submission.csv", "sample_submission.csv"
)
shutil.copyfile(competition_path / "test.csv", "test.csv")
shutil.copyfile(competition_path / "train.csv", "train.csv")
shutil.copyfile(
    competition_path / "sample_submission.csv", "sample_submission.csv"
)
shutil.copyfile(
    competition_path / "data_description.txt", "data_description.txt"
)


['data_description.txt', 'sample_submission.csv', 'test.csv', 'train.csv']


'data_description.txt'

In [ ]:
train_df = pl.read_csv(
    "train.csv",
    #    infer_schema_length=None,
    ignore_errors=True,
)
test_df = pl.read_csv(
    "test.csv",
    #   infer_schema_length=None,
    ignore_errors=True,
)
target_col: pl.Series = train_df.drop_in_place("SalePrice")
df: pl.DataFrame = pl.concat([train_df, test_df])

df = df.with_columns(
    [
        pl.when(pl.col("LotFrontage") == "NA")
        .then(pl.lit(0))
        .otherwise(pl.col("LotFrontage"))
        .cast(pl.UInt16)
        .alias("LotFrontage"),
        pl.when(pl.col("GarageYrBlt") == "NA")
        .then(pl.lit(0))
        .otherwise(pl.col("GarageYrBlt"))
        .cast(pl.UInt16)
        .alias("GarageYrBlt"),
        pl.col("MasVnrArea").fill_null(0),
    ]
)
df_numeric_cols = df.select(pl.selectors.numeric())
df_string_cols = df.select(pl.selectors.string().cast(pl.Categorical))

df_numeric_cols = df_numeric_cols.drop("GarageYrBlt", "Id")
df_numeric_cols = df_numeric_cols.fill_null(0)
df = df_string_cols.hstack(df_numeric_cols)
df.write_parquet("train_test_merged_df.parquet", statistics="full")


In [99]:
str_unique_vals_count = df_string_cols.select(pl.all().n_unique()).transpose(
    include_header=True, header_name="col_name", column_names=["unique_count"]
)
numeric_unique_vals_count = df_numeric_cols.select(
    pl.all().n_unique()
).transpose(
    include_header=True, header_name="col_name", column_names=["unique_count"]
)
(
    str_unique_vals_count.sort("unique_count", descending=True),
    numeric_unique_vals_count.sort("unique_count", descending=True),
)

(shape: (43, 2)
 ┌──────────────┬──────────────┐
 │ col_name     ┆ unique_count │
 │ ---          ┆ ---          │
 │ str          ┆ u32          │
 ╞══════════════╪══════════════╡
 │ Neighborhood ┆ 25           │
 │ Exterior2nd  ┆ 17           │
 │ Exterior1st  ┆ 16           │
 │ SaleType     ┆ 10           │
 │ Condition1   ┆ 9            │
 │ …            ┆ …            │
 │ Utilities    ┆ 3            │
 │ LandSlope    ┆ 3            │
 │ PavedDrive   ┆ 3            │
 │ Street       ┆ 2            │
 │ CentralAir   ┆ 2            │
 └──────────────┴──────────────┘,
 shape: (35, 2)
 ┌──────────────┬──────────────┐
 │ col_name     ┆ unique_count │
 │ ---          ┆ ---          │
 │ str          ┆ u32          │
 ╞══════════════╪══════════════╡
 │ LotArea      ┆ 1951         │
 │ GrLivArea    ┆ 1292         │
 │ BsmtUnfSF    ┆ 1135         │
 │ 1stFlrSF     ┆ 1083         │
 │ TotalBsmtSF  ┆ 1058         │
 │ …            ┆ …            │
 │ YrSold       ┆ 5            │
 │ BsmtFul